In [1]:
# Cell 1: Understat API (xG Data) for last two seasons
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup

def fetch_understat_data(season_start_year: int):
    url = f"https://understat.com/league/EPL/{season_start_year}"
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    # Extract JSON embedded in the page scripts
    scripts = soup.find_all('script')
    data = None
    for script in scripts:
        if 'teamsData' in script.text:
            json_text = script.text.split("JSON.parse('")[1].split("')")[0]
            json_text = json_text.encode('utf8').decode('unicode_escape')
            data = json.loads(json_text)
            break

    if not data:
        print(f"No Understat data found for season {season_start_year}.")
        return pd.DataFrame()

    matches = []
    for team, tdata in data.items():
        for match in tdata['history']:
            matches.append({
                'date': pd.to_datetime(match['date']),
                'team': tdata['title'],
                'xG': float(match['xG']),
                'xGA': float(match['xGA']),
                'result': match['result'],
                'season': season_start_year
            })
    df = pd.DataFrame(matches)
    return df

# Fetch data for last two seasons and concatenate
seasons = [2023, 2024]
understat_dfs = []

for season in seasons:
    print(f"Fetching Understat data for season {season}...")
    df_season = fetch_understat_data(season)
    understat_dfs.append(df_season)

understat_df = pd.concat(understat_dfs, ignore_index=True)
understat_df.to_csv('understat_data_last_two_seasons.csv', index=False)
print(f"Understat data for seasons {seasons} saved with {len(understat_df)} rows.")


Fetching Understat data for season 2023...
Fetching Understat data for season 2024...
Understat data for seasons [2023, 2024] saved with 1520 rows.


In [2]:
import requests
import pandas as pd
from time import sleep
from dotenv import load_dotenv
import os

load_dotenv()


API_KEY = os.getenv('API_KEY')
HEADERS = {'X-Auth-Token': API_KEY}

def fetch_matches_for_season(season_year):
    url = f"https://api.football-data.org/v4/competitions/PL/matches?season={season_year}"
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        print(f"Failed to fetch data for season {season_year}: {response.status_code}")
        return pd.DataFrame()
    data = response.json()
    
    match_list = []
    for match in data.get('matches', []):
        full_time_score = match.get('score', {}).get('fullTime', {})
        home_score = full_time_score.get('home')
        away_score = full_time_score.get('away')
        match_list.append({
            'date': match['utcDate'],
            'home_team': match['homeTeam']['name'],
            'away_team': match['awayTeam']['name'],
            'home_score': home_score,
            'away_score': away_score,
            'status': match['status'],
            'season': season_year
        })
    return pd.DataFrame(match_list)

all_seasons_df = pd.DataFrame()

start_year = 2023
end_year = 2024  # Adjust for current year or last completed season

for year in range(start_year, end_year + 1):
    print(f"Fetching data for season {year}...")
    season_df = fetch_matches_for_season(year)
    all_seasons_df = pd.concat([all_seasons_df, season_df], ignore_index=True)
    sleep(1)  # be polite with API rate limits

# Convert date column to datetime
all_seasons_df['date'] = pd.to_datetime(all_seasons_df['date'])

print(f"Collected data for {len(all_seasons_df)} matches spanning {start_year}-{end_year}.")
all_seasons_df.to_csv('football_data_last_10_years.csv', index=False)

Fetching data for season 2023...
Fetching data for season 2024...
Collected data for 760 matches spanning 2023-2024.


In [ ]:
import pandas as pd

# Load the two CSVs
football_df = pd.read_csv("football_data_last_10_years.csv")
understat_df = pd.read_csv("understat_data_last_two_seasons.csv")

# --- Step 1: Convert 'date' columns to consistent format ---
football_df['date'] = pd.to_datetime(football_df['date'], utc=True).dt.date
understat_df['date'] = pd.to_datetime(understat_df['date'], utc=True).dt.date

# --- Step 2: Split Understat data into home and away ---
understat_home = understat_df.rename(columns={
    'team': 'home_team',
    'xG': 'home_xG',
    'xGA': 'home_xGA',
    'result': 'home_result'
})

understat_away = understat_df.rename(columns={
    'team': 'away_team',
    'xG': 'away_xG',
    'xGA': 'away_xGA',
    'result': 'away_result'
})

# --- Step 3: Merge home Understat data ---
merged_df = pd.merge(
    football_df,
    understat_home[['date', 'home_team', 'home_xG', 'home_xGA', 'home_result']],
    on=['date', 'home_team'],
    how='left'
)

# --- Step 4: Merge away Understat data ---
merged_df = pd.merge(
    merged_df,
    understat_away[['date', 'away_team', 'away_xG', 'away_xGA', 'away_result']],
    on=['date', 'away_team'],
    how='left'
)

# --- Step 5: Save merged CSV ---
merged_df.to_csv("merged_football_understat_last_two_seasons.csv", index=False)

print(f"Merged dataset saved with {len(merged_df)} rows.")


Merged dataset saved with 760 rows.


In [7]:
# Merge football_data_last_10_years.csv with understat_data_last_two_seasons.csv
# - normalizes team names
# - merges on date + normalized team name
# - uses fuzzy matching per-match-date to fill leftover misses

import pandas as pd
import re
from difflib import SequenceMatcher

# ---------- Config ----------
FOOTBALL_CSV = "football_data_last_10_years.csv"
UNDERSTAT_CSV = "understat_data_last_two_seasons.csv"
OUTPUT_CSV = "merged_football_understat_last_two_seasons_clean.csv"
THRESHOLD = 0.99  # fuzzy-match threshold (0-1). Increase to be stricter.

# ---------- Helpers ----------
def normalize_team(name: str) -> str:
    """Lowercase, remove punctuation, remove common suffix tokens, collapse spaces."""
    if pd.isna(name):
        return ""
    s = str(name).lower()
    s = s.replace('&', 'and')
    s = re.sub(r'\(.*?\)', '', s)            # remove parentheses content
    s = re.sub(r'[^a-z0-9\s]', ' ', s)       # keep alphanumeric + space
    # remove common suffix tokens like "fc", "afc", "f c", "f.c" (these can vary)
    s = re.sub(r'\b(fc|afc|f c|f c|f c|f c|f c|f c|f c|f c|f c|f c)\b', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

def best_match(name_norm: str, candidates_norms: list):
    """Return (best_candidate_norm, score) using SequenceMatcher."""
    best_score = 0.0
    best_candidate = None
    for cand in candidates_norms:
        score = SequenceMatcher(None, name_norm, cand).ratio()
        if score > best_score:
            best_score = score
            best_candidate = cand
    return best_candidate, best_score

# ---------- Load data ----------
football_df = pd.read_csv(FOOTBALL_CSV, parse_dates=['date'])
understat_df = pd.read_csv(UNDERSTAT_CSV, parse_dates=['date'])

# Normalize dates to plain date (no tz info) for deterministic matching
football_df['date'] = pd.to_datetime(football_df['date'], utc=True, errors='coerce').dt.date
understat_df['date'] = pd.to_datetime(understat_df['date'], utc=True, errors='coerce').dt.date

# ---------- Normalize names ----------
football_df['home_norm'] = football_df['home_team'].apply(normalize_team)
football_df['away_norm'] = football_df['away_team'].apply(normalize_team)
understat_df['team_norm'] = understat_df['team'].apply(normalize_team)

# ---------- First-pass merge on normalized names ----------
# Prepare trimmed understat with needed columns
under_cols = ['date', 'team_norm', 'xG', 'xGA', 'result']
under_small = understat_df[under_cols].copy()

# Merge home stats
merged = pd.merge(
    football_df,
    under_small,
    left_on=['date', 'home_norm'],
    right_on=['date', 'team_norm'],
    how='left',
    suffixes=('', '_home_tmp')
)
merged = merged.rename(columns={'xG':'home_xG', 'xGA':'home_xGA', 'result':'home_result'})
merged.drop(columns=['team_norm'], inplace=True, errors='ignore')

# Merge away stats (use a fresh copy of under_small)
merged = pd.merge(
    merged,
    under_small,
    left_on=['date', 'away_norm'],
    right_on=['date', 'team_norm'],
    how='left',
    suffixes=('', '_away_tmp')
)
merged = merged.rename(columns={'xG':'away_xG', 'xGA':'away_xGA', 'result':'away_result'})
merged.drop(columns=['team_norm'], inplace=True, errors='ignore')

# ---------- Diagnostics: how many missing after initial merge ----------
missing_home_before = merged['home_xG'].isna().sum()
missing_away_before = merged['away_xG'].isna().sum()
total_rows = len(merged)
print(f"Total rows: {total_rows}")
print(f"Missing home_xG after norm-merge: {missing_home_before}")
print(f"Missing away_xG after norm-merge: {missing_away_before}")

# ---------- Fuzzy-fill for rows still missing (per match date) ----------
# Build a small helper: index understat rows by (date) with lists of team_norm -> row(s)
under_by_date = {}
for d, group in understat_df.groupby('date'):
    under_by_date[d] = group[['team_norm', 'xG', 'xGA', 'result']].to_dict('records')

# Fill home_xG using fuzzy match within same match date candidates
filled_home = 0
for idx, row in merged[merged['home_xG'].isna()].iterrows():
    d = row['date']
    name_norm = row['home_norm']
    candidates = under_by_date.get(d, [])
    if not candidates:
        continue
    candidate_norms = [c['team_norm'] for c in candidates]
    best_cand_norm, best_score = best_match(name_norm, candidate_norms)
    if best_score >= THRESHOLD:
        # find the candidate record with that norm
        rec = next((c for c in candidates if c['team_norm'] == best_cand_norm), None)
        if rec is not None:
            merged.at[idx, 'home_xG'] = rec['xG']
            merged.at[idx, 'home_xGA'] = rec['xGA']
            merged.at[idx, 'home_result'] = rec['result']
            merged.at[idx, '_home_match_score'] = best_score
            filled_home += 1

# Fill away_xG using fuzzy match within same match date candidates
filled_away = 0
for idx, row in merged[merged['away_xG'].isna()].iterrows():
    d = row['date']
    name_norm = row['away_norm']
    candidates = under_by_date.get(d, [])
    if not candidates:
        continue
    candidate_norms = [c['team_norm'] for c in candidates]
    best_cand_norm, best_score = best_match(name_norm, candidate_norms)
    if best_score >= THRESHOLD:
        rec = next((c for c in candidates if c['team_norm'] == best_cand_norm), None)
        if rec is not None:
            merged.at[idx, 'away_xG'] = rec['xG']
            merged.at[idx, 'away_xGA'] = rec['xGA']
            merged.at[idx, 'away_result'] = rec['result']
            merged.at[idx, '_away_match_score'] = best_score
            filled_away += 1

# ---------- Diagnostics after fuzzy fill ----------
missing_home_after = merged['home_xG'].isna().sum()
missing_away_after = merged['away_xG'].isna().sum()
print(f"Filled home_xG via fuzzy: {filled_home}")
print(f"Filled away_xG via fuzzy: {filled_away}")
print(f"Missing home_xG after fuzzy-fill: {missing_home_after}")
print(f"Missing away_xG after fuzzy-fill: {missing_away_after}")

# Show some sample rows where still missing for manual inspection
if missing_home_after + missing_away_after > 0:
    print("\nSample rows still missing xG (for manual inspection):")
    display(merged[merged['home_xG'].isna() | merged['away_xG'].isna()][
        ['date','home_team','away_team','home_xG','away_xG','home_norm','away_norm']
    ].head(10))

# ---------- Housekeeping: drop helper norm columns and internal scores if you like ----------
merged.drop(columns=['home_norm','away_norm'], inplace=True, errors='ignore')

# Optional: keep fuzzy match scores for traceability, or remove them:
# merged.drop(columns=['_home_match_score','_away_match_score'], inplace=True, errors='ignore')

# Save final merged file
merged.to_csv(OUTPUT_CSV, index=False)
print(f"\nMerged & cleaned dataset saved to: {OUTPUT_CSV}")


Total rows: 760
Missing home_xG after norm-merge: 171
Missing away_xG after norm-merge: 171
Filled home_xG via fuzzy: 0
Filled away_xG via fuzzy: 0
Missing home_xG after fuzzy-fill: 171
Missing away_xG after fuzzy-fill: 171

Sample rows still missing xG (for manual inspection):


,date,home_team,away_team,home_xG,away_xG,home_norm,away_norm
2,2023-08-12,AFC Bournemouth,West Ham United FC,1.510250,NaN,bournemouth,west ham united
3,2023-08-12,Brighton & Hove Albion FC,Luton Town FC,NaN,NaN,brighton and hove albion,luton town
7,2023-08-13,Brentford FC,Tottenham Hotspur FC,1.907560,NaN,brentford,tottenham hotspur
13,2023-08-19,Wolverhampton Wanderers FC,Brighton & Hove Albion FC,2.688210,NaN,wolverhampton wanderers,brighton and hove albion
14,2023-08-19,Tottenham Hotspur FC,Manchester United FC,NaN,2.44604,tottenham hotspur,manchester united
17,2023-08-20,West Ham United FC,Chelsea FC,NaN,1.60916,west ham united,chelsea
19,2023-08-25,Chelsea FC,Luton Town FC,3.414190,NaN,chelsea,luton town
20,2023-08-26,AFC Bournemouth,Tottenham Hotspur FC,0.483582,NaN,bournemouth,tottenham hotspur
25,2023-08-26,Brighton & Hove Albion FC,West Ham United FC,NaN,NaN,brighton and hove albion,west ham united
29,2023-09-01,Luton Town FC,West Ham United FC,NaN,NaN,luton town,west ham united



Merged & cleaned dataset saved to: merged_football_understat_last_two_seasons_clean.csv
